In [16]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image,ImageTk
import datetime

# ========== PRODUCTS ==========
products = {"T-SHIRT": 299, "CROPTOP": 399, "JEANS": 799, "KURTI": 199, "SHORTS": 299,"HOODIE": 599}

# Global variable to store logged-in user
current_user = None

# ========== LOGIN SCREEN ==========
def login_screen():
    login = tk.Tk()
    login.title("Login - Clothing Zone")
    login.geometry("400x300")
    login.configure(bg="lightgray")
    

    tk.Label(login, text="LOGIN", font=("Arial", 18, "bold"), bg="lightgray").pack(pady=20)
    tk.Label(login, text="Username:", bg="lightgray").pack()
    entry_user = tk.Entry(login)
    entry_user.pack()
    tk.Label(login, text="Password:", bg="lightgray").pack()
    entry_pass = tk.Entry(login, show="*")
    entry_pass.pack()

    def do_login():
        global current_user
        user = entry_user.get()
        passwd = entry_pass.get()
        if user and passwd:
            current_user = user
            login.destroy()
            open_billing_window()
        else:
            messagebox.showerror("Login Failed", "Enter both username and password")

    tk.Button(login, text="Login", command=do_login, bg="green", fg="white").pack(pady=20)
    login.mainloop()
def open_billing_window():
    wid = tk.Tk()
    wid.title("CLOTHING ZONE")
    wid.geometry("1000x900")
    wid.configure(bg="white")

    bg_img = Image.open("bck.jpg")  
    bg_img = bg_img.resize((900, 900))  
    bg_photo = ImageTk.PhotoImage(bg_img)

    bg_label = tk.Label(wid, image=bg_photo)
    bg_label.image = bg_photo 
    bg_label.place(x=0, y=0, relwidth=1, relheight=1)

    # ==== CLOCK ====
    clock_label = tk.Label(wid, font=("arial", 10), fg="white", bg="black")
    clock_label.place(x=800, y=40)
    def update_clock():
        now = datetime.datetime.now()
        current_time = now.strftime("%H:%M:%S - %d:%m:%Y")
        clock_label.config(text=current_time)
        wid.after(1000, update_clock)
    update_clock()
    def logout():
        confirm = messagebox.askyesno("Logout", "Are you sure you want to logout?")
        if confirm:
            wid.destroy()
            login_screen()

    tk.Button(wid, text="Logout", command=logout, bg="red", fg="white").place(x=900, y=20)

    # ==== CUSTOMER DETAILS ====
    tk.Label(wid, text="CUSTOMER NAME:", bg="white", font=("Arial", 11)).pack()
    entry_name = tk.Entry(wid)
    entry_name.pack()

    tk.Label(wid, text="PHONE NO:", bg="white", font=("Arial", 11)).pack()
    entry_phone = tk.Entry(wid)
    entry_phone.pack()

    # ==== BILLING FRAME ====
    frame = tk.Frame(wid, bg="white", padx=10, pady=5, relief=tk.RIDGE, bd=2)
    frame.place(x=150, y=200)

    tk.Label(frame, text="JEON CLOTHING ZONE", relief=tk.RAISED, font=("TimesNewRoman", 20),
             bg="lightblue", fg="white").grid(row=0, column=0, columnspan=2, pady=20, sticky="nsew")

    quantity_entries = {}
    row = 1
    for item, price in products.items():
        tk.Label(frame, text=f'{item} (Rs.{price})', bg="white", font=("Arial", 11)).grid(row=row, column=0, sticky="w", pady=10)
        entry = tk.Entry(frame, width=5)
        entry.grid(row=row, column=1, pady=5)
        quantity_entries[item] = entry
        row += 1

    # ==== BILL FUNCTIONS ====
    def generate_bill():
        name = entry_name.get()
        phone = entry_phone.get()
        total = 0
        bill_text = ""
        report_text = ""
        now = datetime.datetime.now()
        timestamp = now.strftime("%Y-%m-%d %H:%M:%S")

        for item, entry in quantity_entries.items():
            try:
                qnty = int(entry.get())
                if qnty > 0:
                    price = products[item]
                    subtotal = price * qnty
                    total += subtotal
                    bill_text += f"{item}: {qnty} x Rs.{price} = Rs.{subtotal}\n"
                    report_text += f"{item}:{qnty}x{price}=Rs.{subtotal}\n"
            except ValueError:
                 messagebox.showwarning("Invalid input", f"Enter valid quantity for {item}")
                 return
            

        bill_text += f"\nTotal amount: Rs.{total}\n"
        bill_text += f"\n {name} THANK U FOR SHOPPING!! "
        bill_text += f"\nPhone: {phone}"
        bill_text += f"\n------ TEAM ZUDIO -----"
        bill_text += f"\n-------------------------------"

        # ========  WRITE TO SALES FILE ========
        with open("sales_report.txt", "a") as f:
            f.write(f"User: {current_user}\n")
            f.write(f"Date & Time: {timestamp}\n")
            f.write(f"Customer: {name}, Phone: {phone}\n")
            f.write(report_text)
            f.write(f"Total: Rs.{total}\n")
            f.write("-" * 40 + "\n")

        messagebox.showinfo("YOUR BILL", bill_text)

    def clear_all():
        for entry in quantity_entries.values():
            entry.delete(0, tk.END)

    tk.Button(frame, text="Generate", command=generate_bill, font=("TimesNewRoman", 10),
              bg="green", fg="white").grid(padx=50, pady=10)
    tk.Button(frame, text="Clear All", command=clear_all, font=("TimesNewRoman", 10),
              bg="white", fg="red").grid(column=1, pady=10)
  
    chat_frame = tk.Frame(wid, bd=2, relief=tk.GROOVE, bg="white")
    chat_frame.place(x=700, y=200, width=270, height=400)

    chat_display = tk.Text(chat_frame, height=20, width=33, bg="white", fg="black", wrap=tk.WORD)
    chat_display.pack(padx=5, pady=5)

    chat_input = tk.Entry(chat_frame, width=25)
    chat_input.pack(side=tk.LEFT, padx=5, pady=5)

    def chat_response():
        user_msg = chat_input.get().strip().lower()
        if not user_msg:
            return

        chat_display.insert(tk.END, f"You: {user_msg}\n")

        if "hello" in user_msg or "hi" in user_msg:
            bot_reply = "Hello! How can I help you today?"
        elif "clear" in user_msg:
            bot_reply = "Click on 'Clear All' to reset product entries."
        else:
            bot_reply = "sure fill the details."

        chat_display.insert(tk.END, f"Bot: {bot_reply}\n")
        chat_input.delete(0, tk.END)

    chat_send = tk.Button(chat_frame, text="Send", command=chat_response)
    chat_send.pack(side=tk.RIGHT, padx=5)
    wid.mainloop()

login_screen()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\PRATHIPA\AppData\Local\Programs\Python\Python313\Lib\tkinter\__init__.py", line 2068, in __call__
    return self.func(*args)
           ~~~~~~~~~^^^^^^^
  File "C:\Users\PRATHIPA\AppData\Local\Temp\ipykernel_29544\702970185.py", line 35, in do_login
    open_billing_window()
    ~~~~~~~~~~~~~~~~~~~^^
  File "C:\Users\PRATHIPA\AppData\Local\Temp\ipykernel_29544\702970185.py", line 47, in open_billing_window
    bg_img = Image.open("bck.jpg")
  File "C:\Users\PRATHIPA\AppData\Local\Programs\Python\Python313\Lib\site-packages\PIL\Image.py", line 3513, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: 'bck.jpg'
